# Projet 1 : Alignement des séquences et programmation dynamique
BAKKALI Yahya : 000445166

# Introduction
En bio-informatique, l'alignement de séquences est une manière de représenter deux ou plusieurs séquences de macromolécules biologiques (ADN, ARN ou protéines) les unes sous les autres, de manière à en faire ressortir les régions homologues ou similaires [1]. Dans le cadre de ce projet on étudiera seulement l'alignement par paires et comment réaliser un alignement optimal entre deux séquences de protéines en utilisant la programmation dynamique comme méthode parmi les autres méthodes existantes. La programmation dynamique consiste à résoudre un problème en le décomposant en sous-problèmes, puis à résoudre les sous-problèmes, des plus petits aux plus grands en stockant les résultats intermédiaires [2]. La visualisation des régions de similarités obtenues par l'alignement des séquences permet d'identifier si elles sont la conséquence de la même fonctionnalité ou de la même structure de domaine ou plutôt une relation évolutive entre les deux séquences. Il y en a deux types d'alignement un global qui se restreint sur la totalité de la longueur des deux séquences et l'autre local qui se focalise seulement sur des segments limités de la séquence où le taux de similarité est fort par rapport à la totalité de la séquence. Parmi les algorithmes réalisés pour obtenir les alignements optimaux on trouve deux trop connue celui de Needleman-Wunsch qui réalise l'alignement global et l'autre de Smith-Waterman qui réalise l'alignement local. Le meilleur alignement des séquences correspond au meilleur score possible de correspondance entre deux séquences, les matrices de substitution permettent de calculer ce score. Les matrices de substitution sont des matrices utilisées en bioinformatique pour réaliser des alignements de séquences biologiques reliées évolutivement. Elles permettent de donner un score de similarité ou de ressemblance entre deux acides aminés [3].Il existe plusieurs de ces matrices, basées sur des principes de construction différents. Par exemple la matrice __PAM__ est base sur les mutations observées dans un alignement global avec la conservation des régions mutables tandis que la matrice __BLOSUM__ n'est fondée que sur des segments spécifiques dans la séquence sans l'introduction des trous. Le numéro de PAM indique la divergence entre les séquences et le numéro de BLOSUM indique le pourcentage d'identité.

# Matériel & Méthodes
Pour la réalisation de ce projet on utilisera Python3 comme langage de programmation et des jeux données pour tester et évaluer les algorithmes d'alignement :
1. Deux types de matrices de substitution __PAM__ et __BLOSUM__ stockée dans des fichiers textes blosum62, blosum80,pam60,pam120.
2. Deux séquences au format FASTA __WW-squence.fasta__ pour le global alignement et __protein-sequences.fasta__ pour le local alignement. 

Avant l'extraction des données à partir des fichiers fournies on a besoin du certaines structures pour les stocker.<br> Pour cela on va créer deux ADT(Abstract Data Type) un pour la séquence des acides amines et l'autre pour les matrices de scores et de substitutions.

In [1]:
class ADTsequence(object):

    def __init__(self, sequence):
        self.sequence = sequence

    def __getitem__(self, key):
        return self.sequence[key]

    def __len__(self):
        return len(self.sequence)

    def __str__(self):
        output = ""
        for i in range(len(self) // 60 + 1):
            output += self.sequence[i * 60:(i + 1) * 60]
            output += "\n"
        return output[:-1]

ADTsequence est un ADT qui représente une séquence d’acides aminés et toutes les opérations qu’on peut exécuter sur une séquence qui sont nécessaires pour l’implémentation d’algorithme d’alignement (p.ex. retourner un acide aminé à une certaine position ou visualiser la séquence en format FASTA).

In [2]:
class ADTmatrix:

    def __init__(self, n=None, m=None):
        
        self.n = n
        self.m = m

        if n and m:
            self.matrix = [[0 for i in range(m)] for j in range(n)]
        else:
            self.matrix = []

    def getItem(self, i, j):
        return self.matrix[i][j]

    def setItem(self, i, j, value):
        self.matrix[i][j] = value

    def maxValue(self):

        x, y = 1, 1
        maximum = self.getItem(x, y)
        for i in range(1, self.n):
            for j in range(1, self.m):
                if maximum < self.getItem(i, j):
                    maximum = self.getItem(i, j)
                    x, y = i, j
        return x, y
        
    def addRow(self, row):
        self.matrix.append(row)

    def __str__(self):
        output = ""
        for i in range(len(self.matrix)):
            for j in range(len(self.matrix[0])):
                output += str(self.matrix[i][j])
                output += " "
            output += "\n"
        return output[:-1]

ADTmatrix est un ADT qui représente une matrice et les opérations qu’on peut exécuter sur cette matrice.Il sera utiliser pour créer des sous-classes matrice, pour la matrice de substitution et pour les matrices scores que nous utiliserons lors de l’implémentation de l’algorithme d’alignement de séquences.

In [3]:
class SubstitutionMatrix(ADTmatrix):

    def __init__(self, n=None, m=None):
        super().__init__(n, m)
        self.myList = []

    def __getitem__(self, key):
        return self.myList.index(key)

    def Append(self, value):
        self.myList.append(value)

SubstitutionMatrix est une sous-classe de la classe ADTmatrix dans laquelle on stockera les matrices de substitution qui seront utilisé après dans les algorithmes d'alignement.

In [4]:
def SequencesParser(filename):
    f = open(filename, encoding='utf-8')
    mylist = []
    sequence = ''
    for line in f:
        if line[0] not in [">", " ", "\n"]:
            sequence += line.strip("\n")
        elif len(sequence) :
            mylist.append(ADTsequence(sequence))
            sequence = ''
    if len(sequence) :
        mylist.append(ADTsequence(sequence))
    f.close()
    return mylist

sequenceParser un parser qui peut lire des fichiers avec des séquences (en format FASTA) et qui enregistre chaque séquence comme une instance de l’ADTsequence.

In [5]:
def MatrixParser(filename):
    f = open(filename, encoding='utf-8')
    matrix = SubstitutionMatrix()
    for line in f:
        if line[0] not in ["#", " ", "\n"]:
            matrix.Append(line[0])
            matrix.addRow(line[1:].split())
    f.close()
    return matrix

MatrixParser un parser qui peut lire des fichiers qui représentent des matrices de substitution et qui enregistre chaque
matrice comme une instance de l’ADT substitution matrix.

In [6]:
GLOBALsequences = SequencesParser("WW-sequence.fasta")
LOCALsequences = SequencesParser("protein-sequences.fasta")
pam120 = MatrixParser("pam120.txt")
pam60 = MatrixParser("pam60.txt")
blosum80 = MatrixParser("blosum80.txt")
blosum62 = MatrixParser("blosum62.txt")


Extraction des données et l'initialisation des différentes matrices de substitutions.

In [7]:
class ScoringMatrix(ADTmatrix):

    def __init__(self, character, mode, sequence1, sequence2, I=None, E=None):
        super().__init__(len(sequence1) + 1, len(sequence2) + 1)
        self.sequence1 = sequence1
        self.sequence2 = sequence2
        self.n = len(sequence1) + 1
        self.m = len(sequence2) + 1

        if character == "V":
            self.initV()
        elif character == "W":
            self.initW()
        else:
            self.initS(mode, I, E)

    def initS(self, mode, I, E):

        for i in range(1, self.m):
            gap = - I - (i - 1) * E
            if mode == "LOCAL":
                self.setItem(0, i, max(0, gap))
            else:
                self.setItem(0, i, gap)

        for j in range(1, self.n):
            gap = - I - (j - 1) * E
            if mode == "LOCAL":
                self.setItem(j, 0, max(0, gap))
            else:
                self.setItem(j, 0, gap)

    def initV(self):
        for i in range(self.m):
            self.setItem(0, i, - float("inf"))

        for j in range(1, self.n):
            self.setItem(j, 0, 0)

    def initW(self):
        for i in range(1, self.m):
            self.setItem(0, i, 0)

        for j in range(self.n):
            self.setItem(j, 0, - float("inf"))

ScoringMatrix est une sous-classe de la classe ADTmatrix qui permet de créer trois types d'instance différentes S, V, W. Les matrices V et W ont comme rôle de retenir les pénalités où un trou a déjà été initialisé tandis que S est la matrice utilisée pour construire le parcours. V et W sont initialisé de la façon que chaque matrice respecte ces deux formes respectivement :

|      |      | S    | E    | Q    | 2    |                            
|------|------|------|------|------|------|
|      | -inf | -inf | -inf |-inf  |-inf  |
|   S  | 0    |      |      |      |      |
|   E  | 0    |      |      |      |      |
|   Q  | 0    |      |      |      |      |
|   1  | 0    |      |      |      |      |

                                                      La matrice V
                                                      
|      |      | S    | E    | Q    | 2    |                            
|------|------|------|------|------|------|
|      | -inf | 0    | 0    | 0    | 0    |
|   S  | -inf |      |      |      |      |
|   E  | -inf |      |      |      |      |
|   Q  | -inf |      |      |      |      |
|   1  | -inf |      |      |      |      |

                                                      La matrice W

Par contre pour initialiser la matrice S faut préciser dans quel alignement il sera utilise local ou global parce que dans l'alignement local les valeurs négatives sont remplacées par des zéros. On remplit chaque case par la valeur du gap affine en utilisant la formule suivante $ g(n_{gap}) = - I - (n_{gap} - 1) E $ ou __I__ référence l'insertion du gap et __E__ l'extension du gap.

In [8]:
class NeedlemanWunsch(object):
    def __init__(self, S, V, W, MATSUB, k, I, E):

        self.S = S
        self.V = V
        self.W = W
        self.MATSUB = MATSUB
        self.k = k
        self.I = I
        self.E = E
        self.solutionCounter = 0

        for i in range(1, self.S.n):
            for j in range(1, self.S.m):
                self.fill(i, j)
        self.findAlignment()


    def t(self, i, j):
        x = self.MATSUB[self.S.sequence2[j - 1]]
        y = self.MATSUB[self.S.sequence1[i - 1]]
        return int(self.MATSUB.getItem(x, y))

    def fill(self, i, j):

        self.V.setItem(i, j, max(self.S.getItem(i - 1, j) - self.I, self.V.getItem(i - 1, j) - self.E))
        self.W.setItem(i, j, max(self.S.getItem(i, j - 1) - self.I, self.W.getItem(i, j - 1) - self.E))
        self.S.setItem(i, j, max(self.S.getItem(i - 1, j - 1) + self.t(i, j), self.V.getItem(i,j), self.W.getItem(i,j)))        

La classe NeedlemanWunsch est une implémentation de l'algorithme de Needleman-Wunsch qui est un algorithme qui effectue un alignement global entre deux séquences de protéines en utilisant la programmation dynamique. <br>

La première étape de l'algorithme est de remplir les matrices de scores S, V et W en utilisant les 3 formules ci-dessous:

$$
V(i,j) = max
    \left \{
       \begin{array}{l}
          S(i-1,j)-I \\
          W(i-1,j)-E
       \end{array}
       \right .
$$

$$
W(i,j) = max
    \left \{
       \begin{array}{l}
          S(i,j-1)-I \\
          W(i,j-1)-E
       \end{array}
       \right .
$$

$$
S(i,j) = max
    \left \{
       \begin{array}{l}
          S(i-1,j-1)+t(i,j) \\
          V(i,j) \\
          W(i,j)
       \end{array}
       \right .
$$

La fonction V renvoie la valeur maximale entre l'insertion d'un nouvel espace dans la séquence horizontale et l'extension des espaces dans la séquence verticale.

La fonction W renvoie la valeur maximale entre l'insertion d'un nouvel espace dans la séquence verticale et l'extension des espaces dans la séquence horizontale. 

La fonction S renvoie le maximum entre le match des deux acides amines en utilisant la valeur stocke dans la matrice de substitution et les résultats obtenus auparavant par les deux fonctions V et W à la même position $(i, j)$ .

Dans la classe c'est la méthode fill(i, j) qui s'occupe de remplir les matrices comme c'est expliqué au-dessus et la méthode t(i, j) renvoie la valeur du match entre les deux acides amines

In [9]:
class NeedlemanWunsch(NeedlemanWunsch):
        
    def findAlignment(self, i=None, j=None,seq = "",seq2 = "",seq3 = ""):
        if i is None and j is None:
            i = self.S.n - 1
            j = self.S.m - 1
        if i >= 0 and j >= 0 and self.solutionCounter < self.k :
            if i == 0 and j == 0:
                self.solutionCounter += 1
                show(self.solutionCounter,self.S.getItem(self.S.n - 1, self.S.m - 1), seq[::-1], seq2[::-1], seq3[::-1])

            else:

                if self.S.getItem(i, j) == self.S.getItem(i - 1, j - 1) + self.t(i, j):
                    seq += self.S.sequence1[i - 1]
                    seq2 += self.S.sequence2[j - 1]
                    if self.S.sequence1[i - 1] == self.S.sequence2[j - 1]:
                        seq3 += "|"
                    else:
                        seq3 += ":" if self.t(i, j) >= 0 else "."
                    self.findAlignment(i - 1, j - 1,seq,seq2,seq3)
                    seq = seq[:-1]
                    seq2 = seq2[:-1]
                    seq3 = seq3[:-1]

                if self.S.getItem(i, j) == self.V.getItem(i, j) or j == 0:
                    seq += self.S.sequence1[i - 1]
                    seq2 += "-"
                    seq3 += " "
                    self.findAlignment(i - 1, j,seq,seq2,seq3)
                    seq = seq[:-1]
                    seq2 = seq2[:-1]
                    seq3 = seq3[:-1]

                if self.S.getItem(i, j) == self.W.getItem(i, j) or i == 0:
                    seq += "-"
                    seq2 += self.S.sequence2[j - 1]
                    seq3 += " "
                    self.findAlignment(i, j - 1,seq,seq2,seq3)
                    seq = seq[:-1]
                    seq2 = seq2[:-1]
                    seq3 = seq3[:-1]

Après le remplissage des trois matrices, l'algorithme débute la deuxième phase qui consiste à trouver l'alignement optimal entre les deux séquences de protéines. L'idée de cette phase est de trouver tous les alignements possibles qui ont le même meilleur score, pour cela on commence à la position $ (n, m) $ qui correspond a la derniere case de la matrice S qui contient la valeur maximale qu'on peut obtenir pour un alignement entre ces deux séquences puis en essaye de remonter jusqu'à ce qu'on arrive à la valeur 0 qui se trouvait a la position $ (0,0) $ de la matrice du score S. En remontant la décision d'aller à gauche, en haut ou en diagonale depend d'où vient la valeur à cette position. Si la valeur vient du V on insère un gap dans la séquence horizontale s'il vient du W on l'insert dans la séquence verticale sinon la valeur vient du match et donc on ne fait rien. À un moment donné la valeur peut-être obtenue à partir des deux ou les trois matrices mais on donne toujours la priorité au match parce que c'est plus preferable que d'introduire un gap.

In [10]:
class SmithWaterman(object):
    def __init__(self, S, V, W, MATSUB, l, I, E):

        self.S = S
        self.V = V
        self.W = W
        self.MATSUB = MATSUB
        self.I = I
        self.E = E

        for i in range(1, self.S.n):
            for j in range(1, self.S.m):
                self.fill(i, j)

        i = 0
        while i < l and sum([sum(row) for row in self.S.matrix]):
            indexList = []
            self.topDown(i+1,indexList)
            self.fillZero(indexList)
            x, y = indexList[-1]
            self.computeAgain(x, y)
            i += 1

    def t(self, i, j):
        x = self.MATSUB[self.S.sequence2[j - 1]]
        y = self.MATSUB[self.S.sequence1[i - 1]]
        return int(self.MATSUB.getItem(x, y))

    def fill(self, i, j):

        self.V.setItem(i, j, max(0, self.S.getItem(i - 1, j) - self.I, self.V.getItem(i - 1, j) - self.E))
        self.W.setItem(i, j, max(0, self.S.getItem(i, j - 1) - self.I, self.W.getItem(i, j - 1) - self.E))
        self.S.setItem(i, j, max(0, self.S.getItem(i - 1, j - 1) + self.t(i, j), self.V.getItem(i,j), self.W.getItem(i,j)))

La classe SmithWaterman est une implémentation de l'algorithme de Smith-Waterman qui est un algorithme qui effectue un alignement local entre deux séquences de protéines en utilisant la programmation dynamique. <br>

La première étape de l'algorithme est de remplir les matrices de scores S, V et W en utilisant les 3 formules ci-dessous:

$$
V(i,j) = max
    \left \{
       \begin{array}{l}
          S(i-1,j)-I \\
          W(i-1,j)-E \\
          0
       \end{array}
       \right .
$$

$$
W(i,j) = max
    \left \{
       \begin{array}{l}
          S(i,j-1)-I \\
          W(i,j-1)-E \\
          0
       \end{array}
       \right .
$$

$$
S(i,j) = max
    \left \{
       \begin{array}{l}
          S(i-1,j-1)+t(i,j) \\
          V(i,j) \\
          W(i,j) \\
          0
       \end{array}
       \right .
$$

Les trois fonctions ont le même fonctionnement que celles dans l'alignement global cependant il y a seulement une petite différence par rapport à avant c'est que si la valeur renvoyer et négatifs on la remplace par zéro.

In [11]:
class SmithWaterman(SmithWaterman):
    def topDown(self, number, indexList):

        i, j = self.S.maxValue()
        indexList.append((i, j))
        score = self.S.getItem(i, j)
        seq = ""
        seq2 = ""
        seq3 = ""
        pos = (i,j)
        while self.S.getItem(i, j):

            value = self.S.getItem(i, j)
            pos2 = (i,j)

            if value == self.S.getItem(i - 1, j - 1) + self.t(i, j):
                seq += self.S.sequence1[i - 1]
                seq2 += self.S.sequence2[j - 1]
                if self.S.sequence1[i - 1] == self.S.sequence2[j - 1]:
                    seq3 += "|"
                else:
                    seq3 += ":" if self.t(i, j) >= 0 else "."
                i, j = i - 1, j - 1

            elif value == self.V.getItem(i, j):
                seq += self.S.sequence1[i - 1]
                seq2 += "-"
                seq3 += " "
                i = i - 1

            elif value == self.W.getItem(i, j):
                seq += "-"
                seq2 += self.S.sequence2[j - 1]
                seq3 += " "
                j = j - 1

            indexList.append((i, j))

        show(number, score, seq[::-1], seq2[::-1], seq3[::-1],pos,pos2)
        indexList.pop()

Pour l'alignement local on procède différemment par rapport au global, on ne commence pas à partir de $ (i, j) $ mais à partir de la position de la plus grande valeur de la matrice du score S et s'il y en a plusieurs valeurs maximales on peut prendre n'importe laquelle la seule chose qui va changer c'est juste l'ordre dans lequel notre résultat est obtenu.
Donc la methode maxValue() nous renvoie la position de la valeur maximale de la matrice du score et c'est a partir de cette position qu'on va commencer a construire notre alignement on faisant la meme chose qu'on a fait dans l'alignement global sauf que cette fois ci on s'arrete des qu'on rencontre une valeur 0.

In [12]:
class SmithWaterman(SmithWaterman):
    def fillZero(self, indexList):

        for elem in indexList:
            self.S.setItem(elem[0], elem[1], 0)
            self.V.setItem(elem[0], elem[1], 0)
            self.W.setItem(elem[0], elem[1], 0)

Des qu'on a

In [13]:
class SmithWaterman(SmithWaterman):
    def computeAgain(self, x, y):
        i, j = x, y
        while i < self.S.n:
            while j < self.S.m:
                if self.S.getItem(i,j) > 0 or self.V.getItem(i,j) > 0 or self.W.getItem(i,j) > 0:
                    self.fill(i, j)
                j += 1
            i += 1
            j = y

In [14]:
def show(number, score, sequence, sequence2, sequence3, pos=None, pos2=None):
    print("Soluntion n°: {0} ".format(number))
    w = (len(sequence) // 60 + 1) if len(sequence) % 60 != 0 else len(sequence) // 60
    for i in range(w):
        if pos is None and pos2 is None :
            print(sequence[i * 60:(i + 1) * 60] \
                + "\n" + sequence3[i * 60:(i + 1) * 60] \
                + "\n" + sequence2[i * 60:(i + 1) * 60] + "\n")
        else :
            print(sequence[i * 60:(i + 1) * 60] \
                + " - from {0} to {1}".format(pos2[0],pos[0]) \
                + "\n" + sequence3[i * 60:(i + 1) * 60] \
                + "\n" + sequence2[i * 60:(i + 1) * 60] \
                + " - from {0} to {1}".format(pos2[1],pos[1]) + "\n")

    print("Score :", score)
    print("Identity percentage :", round(sequence3.count("|") / len(sequence3) * 100,2), "%")
    print("Similarity rate:", round((sequence3.count("|") + sequence3.count(":")) / len(sequence3) * 100,2), "%")
    print("-" * 60)

In [15]:
I = 12
E = 2
k = 5
for i in range(len(GLOBALsequences)):
    for j in range(i+1,len(GLOBALsequences)) :
        print("\n" * 2)
        print("Alignment between sequence {0} and {1}".format(i+1,j+1))
        S = ScoringMatrix("S","GLOBAL",GLOBALsequences[i],GLOBALsequences[j],I,E)
        V = ScoringMatrix("V","GLOBAL",GLOBALsequences[i],GLOBALsequences[j])
        W = ScoringMatrix("W","GLOBAL",GLOBALsequences[i],GLOBALsequences[j])
        NeedlemanWunsch(S,V,W,pam120,k,I,E)




Alignment between sequence 1 and 2
Soluntion n°: 1 
VPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRK
.|||:|||..:...|:.|::||.::.|.|:.|..
SPLPPGWEERQDILGRTYYVNHESRRTQWKRPTP

Score : 62
Identity percentage : 38.24 %
Similarity rate: 61.76 %
------------------------------------------------------------



Alignment between sequence 1 and 3
Soluntion n°: 1 
VPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRK
.|||:|||....::|:.:::||..:.|.|:|||.
GPLPPGWEERTHTDGRIFYINHNIKRTQWEDPRL

Score : 76
Identity percentage : 41.18 %
Similarity rate: 67.65 %
------------------------------------------------------------



Alignment between sequence 1 and 4
Soluntion n°: 1 
VPLPAGWEMAKT-SSGQRYFLNHIDQTTTWQDPRK
..||:|||:.:: |||:.|::|||.:::.|:.|..
EKLPPGWEKRMSRSSGRVYYFNHITNASQWERPSG

Score : 67
Identity percentage : 40.0 %
Similarity rate: 71.43 %
------------------------------------------------------------



Alignment between sequence 1 and 5
Soluntion n°: 1 
VPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRK
....:.|...|:::|:.|:.|...:.:||:.|..
SGAKSMWTE

In [17]:
I = 12
E = 2
l = 3
for i in range(len(LOCALsequences)):
    for j in range(i+1,len(LOCALsequences)) :
            print("Alignment between sequence {0} and {1}".format(i+1,j+1))
            S = ScoringMatrix("S","LOCAL",LOCALsequences[i],LOCALsequences[j],I,E)
            V = ScoringMatrix("V","LOCAL",LOCALsequences[i],LOCALsequences[j])
            W = ScoringMatrix("W","LOCAL",LOCALsequences[i],LOCALsequences[j])
            SmithWaterman(S, V, W, blosum62, l, I, E)

Alignment between sequence 1 and 2
Soluntion n°: 1 
LPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRKAMLSQMNVTAPTSPPVQQNMMNSASGPL - from 173 to 264
||.|||:..:.:|:.:|::|..:||||:|||      :::.|.....::.:..|:. |||
LPKGWEVRHAPNGRPFFIDHNTKTTTWEDPR------LKIPAHLRGKTSLDTSNDL-GPL - from 842 to 926

PDGWEQAMTQDGEIYYINHKNKTTSWLDPRLD - from 173 to 264
|.|||:....||:|:||||:.|.|:|.||||:
PPGWEERTHTDGRIFYINHNIKRTQWEDPRLE - from 842 to 926

Score : 194
Identity percentage : 41.3 %
Similarity rate: 65.22 %
------------------------------------------------------------
Soluntion n°: 2 
IPDDVPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDP--------RKAMLSQMNVT----A - from 167 to 263
:|.:..||.|||..:.:.|:.|:::|.::||||..|        .:...||.::.    |
LPTSSGLPPGWEEKQDERGRSYYVDHNSRTTTWTKPTVQATVETSQLTSSQSSAGPQSQA - from 763 to 873

PTSPPVQQ-NMMNSA-SGPLPDGWEQAMTQDGEIYYINHKNKTTSWLDPRL - from 167 to 263
.||...|| ::.::. :|.||.|||...:.:|:.::|:|::|||:|.||||
STSDSGQQVTQPSEIEQGFLPKGWEVRHAPNGRPFFIDHNTKTTTWEDPRL - from 763 to 873

Score : 149
Identity percentage : 3

Soluntion n°: 2 
LPPGWEERQDILGRTYYVNHESRRTQWKRPT - from 612 to 642
|||||::..:..||.||||..:::|.|:||:
LPPGWQSYLSPQGRRYYVNTTTNETTWERPS - from 79 to 109

Score : 89
Identity percentage : 48.39 %
Similarity rate: 74.19 %
------------------------------------------------------------
Soluntion n°: 3 
EIEQGFLPKGWEVRHAPNGRPFFIDHNTKTTTWEDP - from 836 to 871
|.:..:||.||:...:|:||.::::.:|:.||||.|
ESQTVILPPGWQSYLSPQGRRYYVNTTTNETTWERP - from 73 to 108

Score : 83
Identity percentage : 38.89 %
Similarity rate: 69.44 %
------------------------------------------------------------
Alignment between sequence 2 and 6
Soluntion n°: 1 
LPKGWEVRHAPNGRPFFIDHNTKTTTWEDPRL-----KIP----A----H-LRGKTS--- - from 842 to 1315
||:|:|.|.:.:|:.:|:..:|.::||:|||:     .||    |    . |:|.||   
LPEGYEQRTTVQGQVYFLHTQTGVSTWHDPRIPSPSGTIPGGDAAFLYEFLLQGHTSEPR - from 236 to 754

-LDTSN--DLGPLPPGWEERTHTDGRIFYINHNIKRTQWEDPRLENV------------- - from 842 to 1315
 |::.|  :|||||||||.|:.::|||::::||.:.||:.||||:::             
DLNSVNCDELGPLPPGWEV

Alignment between sequence 4 and 6
Soluntion n°: 1 
LPPGWEKRMSRSSGRVYYFNHITNASQWERP - from 7 to 37
||||||.| |..|||:|:.:|.::::|:..|
LPPGWEVR-STVSGRIYFVDHNNRTTQFTDP - from 308 to 337

Score : 76
Identity percentage : 48.39 %
Similarity rate: 74.19 %
------------------------------------------------------------
Soluntion n°: 2 
KLPPGWEKRMSRSSGRVYYFNHITNASQWERPSGNSSSG - from 6 to 44
:||.|:|:| :..:|:||:::..|::|.|:.|...|.||
ELPEGYEQR-TTVQGQVYFLHTQTGVSTWHDPRIPSPSG - from 235 to 272

Score : 63
Identity percentage : 38.46 %
Similarity rate: 69.23 %
------------------------------------------------------------
Soluntion n°: 3 
EEALELINGYIQKI - from 83 to 96
::.||||.|.::||
QKELELIIGGLDKI - from 625 to 638

Score : 30
Identity percentage : 50.0 %
Similarity rate: 78.57 %
------------------------------------------------------------
Alignment between sequence 5 and 6
Soluntion n°: 1 
QTVILPPGWQSYLSPQGRRYYVNTTTNETTWERPSSSPGIPASPGSHRSSL - from 75 to 125
|:..||.|:::..:.||:.|:::|.|:.:||:.|.......:.||:..:

# Références
[1] https://fr.wikipedia.org/wiki/Alignement_de_s%C3%A9quences <br>
[2] https://fr.wikipedia.org/wiki/Programmation_dynamique  <br>
[3] https://fr.wikipedia.org/wiki/Matrice_de_similarit%C3%A9 <br>
<br>
<br>